In [289]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import pandas_ta as ta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
import seaborn as sns

root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()

# feature engineering and scaling
def prep(dataset):
    scaler = StandardScaler()

    macro_path1 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\IRP DATASET  - Copy of US_macroeconomics.csv"
    macro_path2 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\^VIX.csv"
    macro_path3 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\CORESTICKM159SFRBATL.csv"
    macro_path4 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\FFR.csv"
    macro_path5 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GDP.csv"
    macro_path6 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VIX9D_History.csv"
    macro_path7 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\GVZ_History.csv"
    macro_path8 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\OVX_History.csv"
    macro_path9 = r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\raw_data\VVIX_History.csv"
    macro_data1 = pd.read_csv(macro_path1)
    macro_data2 = pd.read_csv(macro_path2)
    macro_data3 = pd.read_csv(macro_path3)
    macro_data4 = pd.read_csv(macro_path4)
    macro_data5 = pd.read_csv(macro_path5)
    macro_data6 = pd.read_csv(macro_path6)
    macro_data7 = pd.read_csv(macro_path7)
    macro_data8 = pd.read_csv(macro_path8)
    macro_data9 = pd.read_csv(macro_path9)

    dataset['Date'] = pd.to_datetime(dataset['Date'], dayfirst=True)
    dataset.set_index('Date', inplace=True)
    dataset['Target'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)

    macro_data1['date'] = pd.to_datetime(macro_data1['date'], dayfirst=True)
    macro_data1.set_index('date', inplace=True)

    macro_data2['Date'] = pd.to_datetime(macro_data2['Date'], dayfirst=True)
    macro_data2.set_index('Date', inplace=True)

    macro_data3['Date'] = pd.to_datetime(macro_data3['Date'], dayfirst=True)
    macro_data3.set_index('Date', inplace=True)

    macro_data4['Date'] = pd.to_datetime(macro_data4['Date'], dayfirst=True)
    macro_data4.set_index('Date', inplace=True)

    macro_data5['Date'] = pd.to_datetime(macro_data5['Date'], dayfirst=True)
    macro_data5.set_index('Date', inplace=True)

    macro_data7['Date'] = pd.to_datetime(macro_data7['Date'], dayfirst=False)
    macro_data7.set_index('Date', inplace=True)

    macro_data8['Date'] = pd.to_datetime(macro_data8['Date'], dayfirst=False)
    macro_data8.set_index('Date', inplace=True)

    macro_data9['Date'] = pd.to_datetime(macro_data9['Date'])
    macro_data9.set_index('Date', inplace=True)

    merge_data1 = dataset.join(macro_data1, how='left').fillna(method='ffill')
    merge_data2 = merge_data1.join(macro_data2, how='left').fillna(method='ffill')
    merge_data3 = merge_data2.join(macro_data3, how='left').fillna(method='ffill')
    merge_data4 = merge_data3.join(macro_data4, how='left').fillna(method='ffill')
    merge_data5 = merge_data4.join(macro_data5, how='left').fillna(method='ffill')
    merge_data7 = merge_data5.join(macro_data7, how='left').fillna(method='ffill')
    merge_data8 = merge_data7.join(macro_data8, how='left').fillna(method='ffill')
    merge_final = merge_data8.join(macro_data9, how='left').fillna(method='ffill')

    merge_final['RSI (14D)'] = ta.rsi(merge_final['Close'], length=14)
    merge_final['20 Day CCI'] = ta.cci(high=merge_final['High'], low=merge_final['Low'], 
                                    close=merge_final['Close'], length=20)
    merge_final['Williams %R'] = ta.willr(high=merge_final['High'], low=merge_final['Low'], 
                                        close=merge_final['Close'], length=14)
    merge_final['EMA (5D)'] = merge_final['Close'].ewm(span=5, adjust=False).mean()
    merge_final['MA50'] = merge_final['Close'].rolling(window=50).mean()

    final_dataset = merge_final.dropna()

    time_shifted = final_dataset#.resample('W-FRI').asfreq().dropna()
    
    features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CPI', 
                'Mortgage_rate', 'Unemp_rate', 'disposable_income','GVZ', 'OVX', 'VVIX', 
                'RSI (14D)', '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'DFF']
    
    time_shifted[features] = time_shifted[features].astype(float)
    time_shifted[features] = scaler.fit_transform(time_shifted[features])

    return time_shifted, scaler

# get dataset
file_path = filedialog.askopenfilename(parent=root, title="Select A File")
ticker = pd.read_csv(file_path)
ticker, scaler = prep(ticker)

ticker

C:\Users\samto\AppData\Local\Temp\ipykernel_16440\658619301.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_shifted[features] = time_shifted[features].astype(float)
C:\Users\samto\AppData\Local\Temp\ipykernel_16440\658619301.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_shifted[features] = scaler.fit_transform(time_shifted[features])


,Open,High,Low,Close,Adj Close,Volume,Target,CPI,Mortgage_rate,Unemp_rate,...,DFF,GDP,GVZ,OVX,VVIX,RSI (14D),20 Day CCI,Williams %R,EMA (5D),MA50
Date,,,,,,,,,,,,,,,,,,,,,
2009-09-18,-1.176076,-1.177533,-1.176933,-1.176824,-1.176824,0.109361,1,-1.482353,1.456424,1.657579,...,-0.585650,14448.882,1.117324,0.270120,-1.493092,1.482107,0.984329,1.006778,-1.180615,-1.208577
2009-09-21,-1.179205,-1.176708,-1.177451,-1.175624,-1.175624,-0.299288,1,-1.482353,1.456424,1.657579,...,-0.591876,14448.882,1.243262,0.254290,-1.455507,1.539945,0.832566,1.066556,-1.178910,-1.206776
2009-09-22,-1.172563,-1.174806,-1.173036,-1.173711,-1.173711,-0.244979,0,-1.482353,1.456424,1.657579,...,-0.591876,14448.882,1.518226,0.194489,-1.624331,1.632955,0.820504,1.069644,-1.177134,-1.205111
2009-09-23,-1.172005,-1.170893,-1.174682,-1.177157,-1.177157,-0.116712,0,-1.482353,1.456424,1.657579,...,-0.591876,14448.882,1.402783,0.244323,-1.629260,1.163946,0.667434,0.589041,-1.177101,-1.203546
2009-09-24,-1.174869,-1.176689,-1.182443,-1.182671,-1.182671,-0.167261,0,-1.482353,1.456424,1.657579,...,-0.598102,14448.882,0.987189,0.423140,-1.469062,0.497239,0.276534,0.107400,-1.178920,-1.202392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-23,2.494212,2.498853,2.523222,2.497305,2.497305,1.383573,0,1.755746,1.341485,-1.004953,...,2.633200,28629.153,-0.389731,-0.605202,-0.376635,-0.304872,-0.509211,-1.056645,2.504898,2.447403
2024-07-24,2.436585,2.411178,2.370387,2.345624,2.345624,2.579649,0,1.755746,1.341485,-1.004953,...,2.633200,28629.153,-0.276387,-0.598753,0.708398,-1.463285,-1.774386,-1.987103,2.453630,2.452126
2024-07-25,2.348285,2.364595,2.305113,2.308410,2.308410,2.166303,1,1.755746,1.341485,-1.004953,...,2.633200,28629.153,-0.368741,-0.619859,0.601189,-1.677050,-2.058419,-1.766556,2.407028,2.455868


In [291]:
# create LSTM model class
class LSTM_Model(nn.Module):
    def __init__(self, input_layer, hidden_layer, output_layer, num_layers=4, dropout=0.3):
        super(LSTM_Model, self).__init__()
        self.hidden_layer = hidden_layer
        self.num_layers = num_layers
        
        # LSTM layers with dropout
        self.lstm = nn.LSTM(input_layer, hidden_layer, num_layers, batch_first=True, dropout=dropout)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Linear layer
        self.linear_layer = nn.Linear(hidden_layer, output_layer)

    def forward(self, input_tensor):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        c0 = torch.zeros(self.num_layers, input_tensor.size(0), self.hidden_layer).to(input_tensor.device)
        
        # Pass through LSTM layers
        out, _ = self.lstm(input_tensor, (h0, c0))
        
        # Apply dropout
        out = self.dropout(out)
        
        # Pass through linear layer
        out = self.linear_layer(out[:, -1, :])
        return out

# create sequences for input data and corresponding labels
def create_sequence(input_data, sequence_length):
    sequences = []
    for i in range(0, len(input_data) - sequence_length):
        sequence = input_data[i : i + sequence_length, :-1]
        label = input_data[i + sequence_length, -1]
        sequences.append((sequence, label))
    return sequences

# train the model with data provided
def trainer(model, train_data, loss_func, opt, epochs, fold, losses):
    model.train()
    for epoch in range(epochs):
        for sequence, labels in train_data:
            opt.zero_grad()
            sequence = sequence.clone().detach().float()
            labels = labels.clone().detach().float().view(-1, 1)
            
            y = model(sequence)
            loss = loss_func(y, labels)
            loss.backward()
            opt.step()

        if epoch % 25 == 1:
            print(f'Epoch {epoch} loss: {loss.item():.4f}')

            new_row = pd.DataFrame({'Fold': [fold+1], 'Epoch': [epoch], 'Loss': [loss.item()]})
            losses = pd.concat([losses, new_row], ignore_index=True)
            
    return losses

def predictor(model, test_data):
    model.eval()
    predictions = []
    with torch.no_grad():
        for sequence, _ in test_data:
            sequence = sequence.clone().detach().float()
            y = model(sequence)
            batch_predictions = torch.sigmoid(y)

            # Convert predictions to tensor if they are scalar or float
            if isinstance(batch_predictions, float) or batch_predictions.ndimension() == 0:
                batch_predictions = torch.tensor([batch_predictions])

            batch_predictions = torch.round(batch_predictions)

            predictions.extend(batch_predictions.squeeze().tolist())
    
    return predictions

# create sequences
sequence_length = 10
sequences = create_sequence(ticker[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'CPI', 
                                    'RSI (14D)', '20 Day CCI', 'Williams %R', 'EMA (5D)', 'MA50', 'Target']].values, sequence_length)

# cross-validation
tscv = TimeSeriesSplit(n_splits=10)
accuracies = []
precisions = []
recalls = []
f1s = []
kappas = []
losses = pd.DataFrame(columns=['Fold', 'Epoch', 'Loss'])

for fold, (train_index, test_index) in enumerate(tscv.split(sequences)):
    print(f'Fold {fold+1}')
    
    train_sequences = [sequences[i] for i in train_index]
    test_sequences = [sequences[i] for i in test_index]
    
    train_data = torch.utils.data.DataLoader(train_sequences, shuffle=True, batch_size=32)
    test_data = torch.utils.data.DataLoader(test_sequences, shuffle=False, batch_size=32)
    
    # initialize model
    model = LSTM_Model(input_layer=12, hidden_layer=10, output_layer=1)
    loss_func = nn.BCEWithLogitsLoss()
    opt = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    # train model
    epochs = 150
    losses = trainer(model, train_data, loss_func, opt, epochs, fold, losses)
    
    # make predictions
    test_labels = [label for _, label in test_sequences]
    predictions = predictor(model, test_data)
    
    # calculate statistics
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, zero_division=0)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)
    kappa = cohen_kappa_score(test_labels, predictions)
    
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    kappas.append(kappa)
    
    print(f'\nAccuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1: {f1:.4f}')
    print(f'Kappa: {kappa:.4f}\n')

# average scores across all folds
avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1s)
avg_kappa = np.mean(kappas)

print(f'\nCross-Validation Results:')
print(f'Average Accuracy: {avg_accuracy:.4f}')
print(f'Average Precision: {avg_precision:.4f}')
print(f'Average Recall: {avg_recall:.4f}')
print(f'Average F1: {avg_f1:.4f}')
print(f'Average Kappa: {avg_kappa:.4f}')


Fold 1
Epoch 1 loss: 0.7155
Epoch 26 loss: 0.6799
Epoch 51 loss: 0.7080
Epoch 76 loss: 0.7167
Epoch 101 loss: 0.7027
Epoch 126 loss: 0.6873

Accuracy: 0.5280
Precision: 0.5280
Recall: 1.0000
F1: 0.6911
Kappa: 0.0000

Fold 2
Epoch 1 loss: 0.6992
Epoch 26 loss: 0.6901
Epoch 51 loss: 0.6609
Epoch 76 loss: 0.7445
Epoch 101 loss: 0.7472
Epoch 126 loss: 0.6651

Accuracy: 0.5575
Precision: 0.5575
Recall: 1.0000
F1: 0.7159
Kappa: 0.0000

Fold 3
Epoch 1 loss: 0.6664
Epoch 26 loss: 0.6811
Epoch 51 loss: 0.6711
Epoch 76 loss: 0.6970
Epoch 101 loss: 0.6654
Epoch 126 loss: 0.7154

Accuracy: 0.5841
Precision: 0.5841
Recall: 1.0000
F1: 0.7374
Kappa: 0.0000

Fold 4
Epoch 1 loss: 0.6914
Epoch 26 loss: 0.6370
Epoch 51 loss: 0.7045
Epoch 76 loss: 0.6603
Epoch 101 loss: 0.7243
Epoch 126 loss: 0.6569

Accuracy: 0.5015
Precision: 0.5015
Recall: 1.0000
F1: 0.6680
Kappa: 0.0000

Fold 5
Epoch 1 loss: 0.7081
Epoch 26 loss: 0.6539
Epoch 51 loss: 0.6984
Epoch 76 loss: 0.6665
Epoch 101 loss: 0.7309
Epoch 126 loss:

In [292]:
fall = (ticker.Target == 0).sum()
rise = (ticker.Target == 1).sum()

rise_per = (rise / (rise + fall)) * 100
fall_per = (fall / (rise + fall)) * 100

print(f'Number of days rising: {rise}')
print(f'Number of days falling: {fall}')

print(f'Rise % is: {rise_per:.2f}%')
print(f'Fall % is: {fall_per:.2f}%')

Number of days rising: 2080
Number of days falling: 1659
Rise % is: 55.63%
Fall % is: 44.37%


In [293]:
results = pd.DataFrame(columns=['Dataset', 'Average Accuracy', 'Average Precision', 
                                'Average Recall', 'Average F1', 'Average Kappa', 'Number of Rises', 
                                'Number of Falls', 'Rise %', 'Fall %'])

datatset_name = (file_path.split('/')[-1].split('.')[0] + '_Monthly')

metrics = {'Dataset': datatset_name ,'Average Accuracy': avg_accuracy, 
           'Average Precision': avg_precision, 'Average Recall': avg_recall, 
           'Average F1': avg_f1, 'Average Kappa': avg_kappa, 'Number of Rises': rise, 'Number of Falls': fall, 
           'Rise %': rise_per, 'Fall %': fall_per}

export_results = results._append(metrics, ignore_index=True)

export_results = pd.concat([export_results, losses], ignore_index=True)

export_results.set_index('Dataset', inplace=True)

export_results.to_csv(r"C:\Users\samto\Desktop\IRP DATA-20240724T195545Z-001\IRP DATA\\" + datatset_name + '_daily.csv')

export_results

,Average Accuracy,Average Precision,Average Recall,Average F1,Average Kappa,Number of Rises,Number of Falls,Rise %,Fall %,Fold,Epoch,Loss
Dataset,,,,,,,,,,,,
^IXIC (3)_Monthly,0.522714,0.56866,0.819788,0.594518,0.001258,2080,1659,55.629848,44.370152,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.715501
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,26,0.679867
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,51,0.707950
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,76,0.716684
...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,26,0.698931
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,51,0.668088
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,76,0.661077
